In [1]:
import cv2
import mediapipe as mp
import socket
import time

In [6]:
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(static_image_mode=False,
                       max_num_hands=1,
                       min_detection_confidence=0.5,
                       min_tracking_confidence=0.5)

# Configurar socket TCP
HOST = "127.0.0.1"
PORT = 5005

client_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
client_socket.connect((HOST, PORT))
print("Conectado al servidor.")

# Ruta del video
VIDEO_PATH = "../unity/two_pinch.mp4"
cap = cv2.VideoCapture(VIDEO_PATH)
if not cap.isOpened():
    print("No se pudo abrir el video.")
    exit()

Conectado al servidor.


In [7]:
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    height, width, _ = frame.shape

    # Convertir a RGB para MediaPipe
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = hands.process(frame_rgb)

    if results.multi_hand_landmarks:
        hand_landmarks = results.multi_hand_landmarks[0]
        data = []

        for lm in hand_landmarks.landmark:
            # Coordenadas en píxeles, y con eje invertido (como en la imagen)
            x_px = int(lm.x * width)
            y_px = int(lm.y * height)
            z_rel = lm.z  # z no está en píxeles, se puede usar como está

            data.append(f"{x_px},{height - y_px},{z_rel:.5f}")

        csv_string = ",".join(data)

        # Enviar por socket
        client_socket.sendall(csv_string.encode("utf-8"))
        print("Frame enviado:", csv_string)

        # Dibujar los puntos
        mp.solutions.drawing_utils.draw_landmarks(
            frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)
        
    #frame = cv2.resize(frame, (0, 0), None, 0.5, 0.5)

    # Mostrar
    cv2.imshow("Hand Tracking", frame)
    if cv2.waitKey(1) & 0xFF == 27:
        break

    time.sleep(0.03)

cap.release()
cv2.destroyAllWindows()
client_socket.close()
print("Conexión cerrada.")

Frame enviado: 989,442,0.00000,1036,479,-0.00975,1066,520,-0.01351,1084,562,-0.01671,1103,595,-0.02000,989,557,-0.00236,974,605,-0.00371,962,636,-0.00643,950,661,-0.00930,950,547,-0.00115,925,592,-0.00210,908,621,-0.00615,893,643,-0.01000,920,529,-0.00118,889,566,-0.00384,869,591,-0.00826,852,608,-0.01194,899,504,-0.00219,862,527,-0.00660,837,543,-0.00969,817,557,-0.01168
Frame enviado: 987,439,0.00000,1030,478,-0.00860,1063,522,-0.01379,1091,565,-0.01931,1110,598,-0.02536,993,556,-0.00557,979,605,-0.01016,967,636,-0.01493,956,660,-0.01870,953,548,-0.00609,928,595,-0.00902,910,623,-0.01323,895,645,-0.01726,921,530,-0.00780,892,569,-0.01135,872,592,-0.01492,856,609,-0.01824,896,507,-0.01058,862,531,-0.01437,838,546,-0.01612,818,560,-0.01755
Frame enviado: 988,440,0.00000,1031,480,-0.00982,1065,526,-0.01520,1090,568,-0.02073,1108,600,-0.02685,993,555,-0.00890,979,605,-0.01535,966,636,-0.02130,955,662,-0.02585,954,546,-0.00942,928,593,-0.01454,909,622,-0.01998,893,645,-0.02477,922,529,-0.

KeyboardInterrupt: 